in 06.1 i defined cohort with atleast 1 SpO2 reading present. This will be the missingness analysis performed on this subset of patients. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0)
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 101 ms


In [3]:
len(patients)

12403

time: 3.59 ms


In [4]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018' 

ventcategory_df=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date)), index_col=0)

vaso_dose_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date)), index_col=0)


echodata_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date)), index_col=0)
labs_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)


weightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date)), index_col=0)
heightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date)), index_col=0)

vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)
uti_all_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date)), index_col=0)
bg_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)

rrt_merged_allpt_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date)), index_col=0)
gcs72_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date)), index_col=0)

sofa_df_72=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 15.5 s


In [5]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
    return(df)

time: 4.44 ms


In [6]:
# subsetting each dataframe to only patients in final_patients_df2_v:
dataframe_list= [
    ventcategory_df,
    vaso_dose_72,
    echodata_72,
    labs_all_nosummary_72,
    weightfirstday_df,
    heightfirstday_df,
    vitals_all_nosummary_72,
    uti_all_72,
    bg_all_nosummary_72,
    rrt_merged_allpt_df,
    gcs72_df,
    sofa_df_72
]

(
ventcategory_df,
 vaso_dose_72,
 echodata_72,
 labs_all_nosummary_72,
 weightfirstday_df,
 heightfirstday_df,
 vitals_all_nosummary_72,
 uti_all_72,
 bg_all_nosummary_72,
 rrt_merged_allpt_df,
 gcs72_df,
 sofa_df_72
) = (df_subset(df) for df in dataframe_list)

time: 1.02 s


In [7]:
labs_all_nosummary_72['icustay_id'].nunique() #15142 

14621

time: 11.8 ms


# task 1: % MISSINGNESS table
need to look for the amount of patients who are missing any value in any of the categories. want to aggregate all variables and % missingness and source into 1 dataframe

In [8]:
print(
    labs_all_nosummary_72['hadm_id'].nunique(), #15142/15207 unique hadm_id/icustay_id
    labs_all_nosummary_72['subject_id'].nunique() #12744/12801 unique patients in this
)

14621 12362
time: 17.7 ms


In [9]:
def missingness_fxn(df,name, groupby,filteron):
    
    """
    input: dataframe w/ clinical data we want to assess the % missing values on.
    output: a dataframe grouped by the different clinical variables with 
        associated % missingnss (ie how many icustays do not have this value)
        
    """

    missing_df= pd.DataFrame(
        100* (1-(df.groupby(groupby)[filteron].nunique()/ len(hadm_id)))) #number of unique icustays with data /total # icustay
    missing_df.reset_index(inplace=True)
    missing_df= missing_df.rename(index=str, columns={"label":'label', filteron:'%missingness'})
    missing_df['source']= name
    missing_df['data_label']=groupby
    return(missing_df)

time: 19.4 ms


#### all dataframes to do %missingness on (add x when complete)
- vaso_dose_72 : x
- ventcategory_df : x
- echodata_72 : ?
- labs_all_nosummary_72 : x
- weightfirstday_df : x
- heightfirstday_df : x
- vitals_all_nosummary_72 :x
- uti_all_72 :x
- bg_all_nosummary_72 :x
- rrt_merged_allpt_df :x
- gcs72_df : x
- sofa_df_72: 

In [10]:
missingness_df= pd.DataFrame()

time: 893 µs


### labs x

In [11]:
print(
    (labs_all_nosummary_72['hadm_id'].nunique(), "out of", len(hadm_id)), #15142/14668 unique hadm_id/icustay_id
    (labs_all_nosummary_72['subject_id'].nunique(), "out of",len(patients)) #12744/12403 unique patients in this
)

(14621, 'out of', 14668) (12362, 'out of', 12403)
time: 13.6 ms


In [12]:
#na values
labs_all_nosummary_72.loc[labs_all_nosummary_72['valuenum'].isna(),:]

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,t_0,delta
839176,13940,137769,298457,2192-12-12 02:20:00,INR,NaN,2192-12-12,0 days 02:20:00.000000000


time: 16 ms


In [13]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(labs_all_nosummary_72,'labs_all_nosummary_72', 'label','icustay_id')]
)

time: 173 ms


### vaso_dose x

In [14]:
print(
    vaso_dose_72['icustay_id'].nunique(), #4777/15207 unique icustay_id/icustay_id
)

4762
time: 2.16 ms


In [15]:
vaso_dose_72.head(2)

,icustay_id,starttime,endtime,vaso_rate,vaso_amount,label,t_0,delta
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,epinephrine,2139-06-02,0 days 16:31:00.000000000
108,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,epinephrine,2139-09-23,0 days 15:07:00.000000000


time: 9.74 ms


In [16]:
vaso_dose_72.loc[vaso_dose_72['vaso_rate'].isna(),:]

,icustay_id,starttime,endtime,vaso_rate,vaso_amount,label,t_0,delta


time: 6.2 ms


In [17]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(vaso_dose_72,'vaso_dose_72', 'label','icustay_id')]
)

time: 33.7 ms


### ventcategory_df x

In [18]:
ventcategory_df.head()

,icustay_id,t_0,vent_day1,vent_day2,vent_day3
0,267059.0,2182-11-12,Mech,None,None
1,278729.0,2159-11-08,Oxygen,Mech,None
2,219670.0,2190-08-01,Oxygen,None,None
3,295185.0,2154-11-11,Oxygen,None,None
4,211472.0,2193-12-11,None,None,None


time: 8.14 ms


In [19]:
#need to reshape the dataframe
ventcategory_df_melt = pd.melt(ventcategory_df, id_vars=['icustay_id','t_0'], var_name='label')

print(
    ventcategory_df_melt['icustay_id'].nunique(), #15207/15207 unique icustay_id/icustay_id
)

14668
time: 9.9 ms


In [20]:
ventcategory_df_melt.head(2)

,icustay_id,t_0,label,value
0,267059.0,2182-11-12,vent_day1,Mech
1,278729.0,2159-11-08,vent_day1,Oxygen


time: 7.13 ms


In [21]:
#na values
ventcategory_df_melt.loc[ventcategory_df_melt['value'].isnull()]

,icustay_id,t_0,label,value


time: 6.74 ms


In [22]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(ventcategory_df_melt,'ventcategory_df', 'label','icustay_id')]
)

time: 15 ms


### echodata_72

In [23]:
echodata_72.head(2)

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality,t_0,delta
0,59653,31038,174978.0,2102-06-14 00:00:00,2102-06-14 13:30:00,Endocarditis.,74.0,165.0,2.00,102/72,102.0,72.0,86.0,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate,2102-06-13,1 days 13:30:00.000000000
1,59654,70150,156140.0,2191-05-10 00:00:00,2191-05-10 15:08:00,"CHF, A-FIB",66.0,263.0,2.25,107/81,107.0,81.0,95.0,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal,2191-05-10,0 days 15:08:00.000000000


time: 17.1 ms


In [24]:
#num of icustay_id/hadm_id
print(
    echodata_72['hadm_id'].nunique(), #5421/15207 unique hadm_id/icustay_id
)

5310
time: 1.66 ms


In [25]:
#number of na values

pd.DataFrame(
    {
    'bsa':[echodata_72.loc[echodata_72['bsa'].isna(),'hadm_id'].count()], 
    'bp': [echodata_72.loc[echodata_72['bp'].isna(),'hadm_id'].count()], 
    'bpsys': [echodata_72.loc[echodata_72['bpsys'].isna(),'hadm_id'].count()], 
    'bpdias': [echodata_72.loc[echodata_72['bpdias'].isna(),'hadm_id'].count()],
    'hr': [echodata_72.loc[echodata_72['hr'].isna(),'hadm_id'].count()],
    'test': [echodata_72.loc[echodata_72['test'].isna(),'hadm_id'].count()]
             }
            )

,bp,bpdias,bpsys,bsa,hr,test
0,506,518,518,1088,700,0


time: 13.3 ms


# NEED TO ASK ABOUT WHICH VALUES ARE MOST IMPORTANT IN THIS

In [26]:
# missingness_df = pd.concat(
#     [missingness_df,
#         missingness_fxn(labs_all_nosummary_72,'labs_all_nosummary_72', 'label','icustay_id')]
# )

time: 896 µs


### weightfirstday_df x

In [27]:
#delete once done debugging
weightfirstday_df=pd.read_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date), index_col=0)

time: 23.8 ms


In [28]:
#need to reshape the df

weightfirstday_df_melt= pd.melt(weightfirstday_df, id_vars='icustay_id', var_name='label')
#weightfirstday_df_melt

time: 7.04 ms


In [29]:
print(
    weightfirstday_df_melt['icustay_id'].nunique(), #15207/15207 unique icustay_id/icustay_id
)

15207
time: 2.81 ms


In [30]:
#na values
weightfirstday_df_melt.loc[weightfirstday_df_melt['value'].isna(),'icustay_id'].count() #1522/15207 do not have any weight, 39522 total null rows.

39522

time: 4.06 ms


In [31]:
#missingness df
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(weightfirstday_df_melt.loc[weightfirstday_df_melt['value'].notnull(),:],'weightfirstday_df', 'label','icustay_id')]
)

time: 16.9 ms


In [32]:
missingness_df

,label,%missingness,source,data_label
0,ALBUMIN,53.470139,labs_all_nosummary_72,label
1,ANION GAP,0.552223,labs_all_nosummary_72,label
2,BANDS,82.881102,labs_all_nosummary_72,label
3,BICARBONATE,0.477229,labs_all_nosummary_72,label
4,BILIRUBIN,44.389146,labs_all_nosummary_72,label
5,BUN,0.463594,labs_all_nosummary_72,label
6,CHLORIDE,0.449959,labs_all_nosummary_72,label
7,CREATININE,0.449959,labs_all_nosummary_72,label
8,GLUCOSE,0.436324,labs_all_nosummary_72,label
9,HEMATOCRIT,0.490864,labs_all_nosummary_72,label


time: 15.8 ms


### heightfirstday_df x

In [33]:
heightfirstday_df.head(2)

,icustay_id,height,height_chart,height_echo
4,216859,172.72,NaN,172.72
5,203487,185.42,185.42,182.88


time: 9.94 ms


In [34]:
#need to reshape the df
heightfirstday_df_melt= pd.melt(heightfirstday_df, id_vars='icustay_id', var_name='label')

time: 5.91 ms


In [35]:
print(
    heightfirstday_df_melt['icustay_id'].nunique(), #15207/15207 unique icustay_id/icustay_id
)

14668
time: 2.57 ms


In [36]:
#na values
print(
    heightfirstday_df_melt.loc[heightfirstday_df_melt['value'].isna(),'icustay_id'].count(), #22776 total null rows.
    heightfirstday_df.loc[heightfirstday_df['height'].isna(),'icustay_id'].count() #5212/15207 do not have any weight,
)

21915 5010
time: 3.63 ms


In [37]:
#missingness df
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(heightfirstday_df_melt.loc[heightfirstday_df_melt['value'].notnull(),:],'heightfirstday_df', 'label','icustay_id')]
)

time: 11.6 ms


In [38]:
missingness_df

,label,%missingness,source,data_label
0,ALBUMIN,53.470139,labs_all_nosummary_72,label
1,ANION GAP,0.552223,labs_all_nosummary_72,label
2,BANDS,82.881102,labs_all_nosummary_72,label
3,BICARBONATE,0.477229,labs_all_nosummary_72,label
4,BILIRUBIN,44.389146,labs_all_nosummary_72,label
5,BUN,0.463594,labs_all_nosummary_72,label
6,CHLORIDE,0.449959,labs_all_nosummary_72,label
7,CREATININE,0.449959,labs_all_nosummary_72,label
8,GLUCOSE,0.436324,labs_all_nosummary_72,label
9,HEMATOCRIT,0.490864,labs_all_nosummary_72,label


time: 12.7 ms


### vitals_all_nosummary_72 x

In [39]:
vitals_all_nosummary_72.head(2)

,subject_id,hadm_id,icustay_id,charttime,vitalid,valuenum,t_0,delta
5,9,150750,220597,2149-11-13 15:20:00,MeanArtPress,61.0,2149-11-13,0 days 15:20:00.000000000
6,9,150750,220597,2149-11-14 11:00:00,MeanArtPress,103.0,2149-11-13,1 days 11:00:00.000000000


time: 10.9 ms


In [40]:
#na values
vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

#why do i have null vitalid's with actual values? need to dive into the sql.
###it looks like it may just extract a lot of values as null that are not relevant, can filter these out

time: 1.67 s


In [41]:
print(
    vitals_all_nosummary_nonull['icustay_id'].nunique(), #14714/15207 unique icustay_id/icustay_id
)

14668
time: 68.8 ms


In [42]:
vitals_all_nosummary_nonull

,subject_id,hadm_id,icustay_id,charttime,vitalid,valuenum,t_0,delta
5,9,150750,220597,2149-11-13 15:20:00,MeanArtPress,61.000000,2149-11-13,0 days 15:20:00.000000000
6,9,150750,220597,2149-11-14 11:00:00,MeanArtPress,103.000000,2149-11-13,1 days 11:00:00.000000000
7,9,150750,220597,2149-11-14 12:00:00,MeanArtPress,134.000000,2149-11-13,1 days 12:00:00.000000000
12,9,150750,220597,2149-11-13 07:00:00,MeanArtPress,85.000000,2149-11-13,0 days 07:00:00.000000000
13,9,150750,220597,2149-11-14 02:00:00,MeanArtPress,96.000000,2149-11-13,1 days 02:00:00.000000000
16,9,150750,220597,2149-11-13 10:00:00,MeanArtPress,92.000000,2149-11-13,0 days 10:00:00.000000000
18,9,150750,220597,2149-11-13 00:30:00,MeanArtPress,104.000000,2149-11-13,0 days 00:30:00.000000000
19,9,150750,220597,2149-11-13 17:00:00,MeanArtPress,84.000000,2149-11-13,0 days 17:00:00.000000000
29,9,150750,220597,2149-11-13 18:00:00,MeanArtPress,93.000000,2149-11-13,0 days 18:00:00.000000000
46,9,150750,220597,2149-11-13 04:00:00,MeanArtPress,104.000000,2149-11-13,0 days 04:00:00.000000000


time: 67 ms


In [43]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(vitals_all_nosummary_nonull.rename(index=str, columns={'vitalid':'label'}),'vitals_all_nosummary_72', 'label','icustay_id')]
)

time: 5.73 s


### uti_all_72 x

In [44]:
uti_all_72=pd.read_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date), index_col=0)

time: 514 ms


In [45]:
uti_all_72.head(2)

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
42,26,197661.0,51486,2126-05-10 14:59:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99604,2126-05-10,0 days 14:59:00.000000000
43,26,197661.0,51486,2126-05-10 14:59:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,4271,2126-05-10,0 days 14:59:00.000000000


time: 13.5 ms


In [46]:
#uti_all_72_melt= pd.melt(uti_all_72, id_vars=['subject_id','hadm_id','itemid','charttime','t_0','delta'], var_name='label')

time: 428 µs


In [47]:
uti_all_72_melt= pd.melt(
    uti_all_72[['hadm_id','charttime','value','valuenum','valueuom','label','fluid']],
    id_vars=['hadm_id','charttime'], var_name='label')

time: 247 ms


In [48]:
uti_all_72_melt_notnull= uti_all_72_melt[uti_all_72_melt['value'].notnull()] #1320954

time: 77.7 ms


In [49]:
uti_all_72_melt_notnull['label'].value_counts()

label    220159
value    220159
fluid    220159
Name: label, dtype: int64

time: 80.5 ms


In [50]:
print(
    uti_all_72_melt_notnull.nunique(), #6145/15207 unique hadm_id/hadm_id
)

hadm_id      6145
charttime    7795
label           3
value          11
dtype: int64
time: 275 ms


In [51]:
#na values
#uti_all_72_melt.loc[uti_all_72_melt['value'].isna(),:]#.count() #220159 na valuenum, 440318 when melted
#uti_all_72_melt[uti_all_72_melt['value'].isnull()]['hadm_id'].count() #440318

time: 615 µs


In [52]:
#uti_all_72_melt[uti_all_72_melt['value'].notnull()]

time: 415 µs


In [53]:
missingness_fxn(uti_all_72_melt_notnull,'uti_all_72_melt', 'label','hadm_id')

,label,%missingness,source,data_label
0,fluid,58.106081,uti_all_72_melt,label
1,label,58.106081,uti_all_72_melt,label
2,value,58.106081,uti_all_72_melt,label


time: 117 ms


In [54]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(uti_all_72_melt_notnull,'uti_all_72_melt', 'label','hadm_id')]
)

time: 83.2 ms


### bg_all_nosummary_72 x

In [55]:
bg_all_nosummary_72=pd.read_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date), index_col=0)

time: 1.38 s


In [56]:
#na values- removing all null values from value.
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]

bg_all_nosummary_72.head(2)

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,unique_var,t_0,delta
82,9,150750,220597,2149-11-13 03:34:00,BASEEXCESS,NaN,-1,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
83,9,150750,220597,2149-11-13 03:34:00,CALCIUM,1.05,1.05,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000


time: 110 ms


In [57]:
print(
    bg_all_nosummary_72['icustay_id'].nunique(), #11244/15207 unique icustay_id/icustay_id
)

11244
time: 6.85 ms


In [58]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(bg_all_nosummary_72,'bg_all_nosummary_72', 'label','icustay_id')]
)

time: 121 ms


In [100]:
bg_all_nosummary_72

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,unique_var,t_0,delta
82,9,150750,220597,2149-11-13 03:34:00,BASEEXCESS,NaN,-1,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
83,9,150750,220597,2149-11-13 03:34:00,CALCIUM,1.05,1.05,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
84,9,150750,220597,2149-11-13 03:34:00,LACTATE,1.80,1.8,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
85,9,150750,220597,2149-11-13 03:34:00,PCO2,34.00,34,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
86,9,150750,220597,2149-11-13 03:34:00,PH,7.42,7.42,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
87,9,150750,220597,2149-11-13 03:34:00,PaO2,68.00,68,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
88,9,150750,220597,2149-11-13 03:34:00,SPECIMEN,NaN,ART,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
89,9,150750,220597,2149-11-13 03:34:00,TEMPERATURE,37.20,37.2,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
90,9,150750,220597,2149-11-13 03:34:00,TOTALCO2,23.00,23,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
91,9,150750,220597,2149-11-13 08:07:00,BASEEXCESS,NaN,-1,2205972149-11-13 08:07:00,2149-11-13,0 days 08:07:00.000000000


time: 34.2 ms


In [59]:
#missingness_fxn(bg_all_nosummary_72,'bg_all_nosummary_72', 'label','icustay_id')

time: 709 µs


### rrt_merged_allpt_df x

In [60]:
rrt_merged_allpt_df.head(10)

,icustay_id,t_0,first_charttime,rrt
0,267059.0,2182-11-12,NaN,NaN
1,278729.0,2159-11-08,NaN,NaN
2,219670.0,2190-08-01,NaN,NaN
3,295185.0,2154-11-11,NaN,NaN
4,211472.0,2193-12-11,NaN,NaN
5,259012.0,2120-05-24,NaN,NaN
7,252135.0,2166-04-13,2166-04-13 00:00:00,1.0
8,293255.0,2167-10-21,2167-10-21 04:31:00,1.0
9,261279.0,2167-10-26,2167-10-26 01:00:00,1.0
10,274352.0,2132-07-28,NaN,NaN


time: 9.4 ms


In [61]:
#na values
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]

time: 2.19 ms


In [62]:
print(
    rrt_merged_allpt_df_notnull['icustay_id'].nunique(), #1223/15207 unique icustay_id/icustay_id
)

1220
time: 1.45 ms


In [63]:
#missingness_fxn(rrt_merged_allpt_df_notnull.rename(index=str,columns={'rrt':"label"}),'rrt_merged_allpt_df_notnull', 'label','icustay_id')

time: 696 µs


In [64]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(rrt_merged_allpt_df_notnull.rename(index=str,columns={'rrt':"label"}),'rrt_merged_allpt_df_notnull', 'label','icustay_id')]
)

time: 36.1 ms


### gcs72_df x

In [65]:
gcs72_df.head(2)

,subject_id,hadm_id,icustay_id,day,mingcs,gcsmotor,gcsverbal,gcseyes,endotrachflag,ICU_admit,t_0,admit_plus_day,delta
0,28448,177527,200012,1.0,15.0,6.0,5.0,4.0,0.0,2153-12-23 05:12:55,2153-12-23,2153-12-24 05:12:55,1 days 05:12:55.000000000
1,9514,127229,200014,2.0,13.0,6.0,3.0,4.0,0.0,2105-02-16 23:16:48,2105-02-17,2105-02-18 23:16:48,1 days 23:16:48.000000000


time: 12.4 ms


In [66]:
#need to reshape the dataframe
gcs72_df_melt = pd.melt(gcs72_df, id_vars=['subject_id','hadm_id','icustay_id','t_0','ICU_admit','admit_plus_day','delta'], var_name='label')

time: 22.8 ms


In [67]:
gcs72_df_melt_notnull= gcs72_df_melt[gcs72_df_melt['value'].notnull()] #62772 ->62650

time: 6.91 ms


In [68]:
print(
    gcs72_df_melt_notnull['icustay_id'].nunique(), #10462/15207 unique icustay_id/icustay_id
)

10405
time: 2.92 ms


In [69]:
missingness_fxn(gcs72_df_melt_notnull,'gcs72_df_melt_notnull', 'label','icustay_id')

,label,%missingness,source,data_label
0,day,29.063267,gcs72_df_melt_notnull,label
1,endotrachflag,29.063267,gcs72_df_melt_notnull,label
2,gcseyes,29.165530,gcs72_df_melt_notnull,label
3,gcsmotor,29.397328,gcs72_df_melt_notnull,label
4,gcsverbal,29.445050,gcs72_df_melt_notnull,label
5,mingcs,29.063267,gcs72_df_melt_notnull,label


time: 28.7 ms


In [70]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(gcs72_df_melt_notnull,'gcs72_df_melt_notnull', 'label','icustay_id')]
)

time: 9.69 ms


### sofa_df_72

In [71]:
sofa_df_72.head(2)

,subject_id,hadm_id,icustay_id,day,sofa,respiration,pao2fio2_vent_min,pao2fio2_novent_min,coagulation,platelet_min,...,meanbp_min,cns,mingcs,renal,creatinine_max,urineoutput,t_0,ICU_admit,approx_charttime,floor_charttime
4,9,150750,220597,4.0,6,0.0,204.0,NaN,0.0,306.0,...,54.0,NaN,NaN,2.0,2.0,6020.0,2149-11-13,2149-11-09 13:07:02,2149-11-13 13:07:02,2149-11-13 00:00:00
5,9,150750,220597,5.0,7,NaN,NaN,NaN,NaN,NaN,...,91.0,NaN,NaN,4.0,NaN,100.0,2149-11-13,2149-11-09 13:07:02,2149-11-14 13:07:02,2149-11-14 00:00:00


time: 19.6 ms


In [72]:
#need to reshape the dataframe
sofa_df_melt = pd.melt(sofa_df_72, id_vars=['subject_id','hadm_id','icustay_id','t_0','day','ICU_admit','approx_charttime','floor_charttime'], var_name='label')

time: 180 ms


In [73]:
sofa_df_melt_notnull= sofa_df_melt[sofa_df_melt['value'].notnull()] #606651 ->302459

time: 28.2 ms


In [74]:
len(sofa_df_melt_notnull)

300386

time: 2.55 ms


In [75]:
print(
    sofa_df_melt_notnull['icustay_id'].nunique(), #13323/15207 unique icustay_id/icustay_id
)

13150
time: 20 ms


In [76]:
missingness_df = pd.concat(
    [missingness_df,
        missingness_fxn(sofa_df_melt_notnull,'sofa_df_melt_notnull', 'label','icustay_id')]
)

time: 72.5 ms


In [77]:
missingness_df_v = missingness_df.sort_values('%missingness', ascending=False)

date= '01102018'

pd.DataFrame(missingness_df_v).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/interim/%s_missingness_df_v.csv' %(date))

time: 7.61 ms


In [78]:
missingness_df

,label,%missingness,source,data_label
0,ALBUMIN,53.470139,labs_all_nosummary_72,label
1,ANION GAP,0.552223,labs_all_nosummary_72,label
2,BANDS,82.881102,labs_all_nosummary_72,label
3,BICARBONATE,0.477229,labs_all_nosummary_72,label
4,BILIRUBIN,44.389146,labs_all_nosummary_72,label
5,BUN,0.463594,labs_all_nosummary_72,label
6,CHLORIDE,0.449959,labs_all_nosummary_72,label
7,CREATININE,0.449959,labs_all_nosummary_72,label
8,GLUCOSE,0.436324,labs_all_nosummary_72,label
9,HEMATOCRIT,0.490864,labs_all_nosummary_72,label


time: 25.7 ms


In [86]:
missingness_df[missingness_df['%missingness']>30.0].sort_values('%missingness', ascending=True)

,label,%missingness,source,data_label
16,PH,31.803927,bg_all_nosummary_72,label
0,height,34.155986,heightfirstday_df,label
26,TOTALCO2,34.776384,bg_all_nosummary_72,label
1,BASEEXCESS,34.776384,bg_all_nosummary_72,label
14,PCO2,34.783202,bg_all_nosummary_72,label
11,LACTATE,37.142078,bg_all_nosummary_72,label
12,LACTATE,38.219253,labs_all_nosummary_72,label
4,BILIRUBIN,44.389146,labs_all_nosummary_72,label
2,height_echo,49.734115,heightfirstday_df,label
2,weight_daily,51.288519,weightfirstday_df,label


time: 21.7 ms


### visualizations

In [79]:
"""
    get the # of filled vs missign values for each variable for our cohort.
    Args:
    -----
    df: dataframe
        name of dataframe initialized in this workbook
    valuenum_col: string
        name of column with the numerical or text variable in question
    label_col: string
        name of column with the lab value label in question
    Returns:
    -----
    filled_summary: df
        list of all variables with # filled values
    missing_summary: df
        list of all variables with # missing values
    percent_summary: df
        list of all variables with # filled values/ # total patients
    """

def missing_fxn(df, valuenum_col, label_col):
    if 'icustay_id' in list(df):
        df= df[df['icustay_id'].isin(list(cohort_df['icustay_id'].unique()))] #good
    else:
        df= df[df['subject_id'].isin(
            list(cohort_df['subject_id'].unique()))] 

    df_notnull = df[df[valuenum_col].notnull()]
    df_null = df[df[valuenum_col].isnull()]
    df_filtered = df.drop_duplicates(['subject_id',label_col])[label_col].value_counts()
    
    filled_summary= df_notnull.drop_duplicates(['subject_id',label_col])[label_col].value_counts()
    missing_summary = df_null.drop_duplicates(['subject_id',label_col])[label_col].value_counts()
    percent_summary = 100*(filled_summary / df_filtered)
    
    #missing_plot=(missing_summary.sort_index().plot.bar(title = f'# of missing values in {df}'))
    
    #filled_summary, missing_summary, percent_summary, 
    return(filled_summary, missing_summary, percent_summary)

time: 21.6 ms


In [80]:
var1, var2, var3,  = missing_fxn(labs_all48, 'valuenum', 'label')
fig, axs = plt.subplots(1,2,figsize=(10,4))
ax1= (var2.sort_index().plot.bar(ax=axs[0], title = '# of missing values in labs_all48'))
ax2= (var3.sort_index().plot.bar(color='grey',
                                ax=axs[1],
                                ylim=(min(var3)-10 ,100),
                                title = '% of values filled in labs_all48'))

NameError: name 'labs_all48' is not defined

time: 181 ms


#### histogram of frequency over each hour.

In [ ]:
def hours_fxn(td):
    return ((td.days*24*3600) + td.seconds)//3600

def days_fxn(td):
    return td.days

In [ ]:
pd.to_timedelta(vaso_dose_72['delta'].head())

In [ ]:
for element in vaso_dose_72['label'].unique():
    print(element)

In [ ]:
size=11
fig=plt.figure(figsize=(size, size*(2/3)), dpi= 80, facecolor='w')
ax = plt.axes()

vaso_dose_72['hours']=pd.to_timedelta(vaso_dose_72['delta']).map(hours_fxn) #good

for element in vaso_dose_72['label'].unique():    
    (vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'hours']
         .value_counts(sort=False)
         .sort_index()
         .plot(kind='bar', sort_columns=True))

In [ ]:
for element in vaso_dose_72['label'].unique():
    size=9
    fig=plt.figure(figsize=(size, size*(2/3)), dpi= 80, facecolor='w')
    ax = plt.axes()
    (vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'hours']
         .value_counts(sort=False)
         .sort_index()
         .plot(kind='bar', sort_columns=True,color='blue'))
    print(element)
    ax.set_ylabel('frequency_for_all_pt', fontsize='large')
    ax.set_xlabel('hours_after_t_0', fontsize='large')
    ax.set_title('%s'%(element), fontsize='large')
    plt.show()

In [ ]:
# #looking at overall missingness

# for element in vaso_dose_72['label'].unique():
#     print(element)
#     print(vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'icustay_id']
#          .nunique())
#     print(((vaso_dose_72.loc[vaso_dose_72.loc[:,'label']==element, 'icustay_id']
#          .nunique())/len(icustay_id))*100,"% of icustay_id's with any value") 

In [ ]:
# print(vaso_dose_72['icustay_id'].nunique())

In [ ]:
# len(icustay_id)

In [ ]:

11/1/18: After looking at the missingness % of patients without a given reading of each data over first 3 days of icustay, we decided that patients without an single SpO2 reading are disqualified from study. 



 
Total: 14668


10/28/18:

# Goal: to make a new icustay_id cohort with the added criteria that patients have at minimum one Sp02 reading during their first 3 days of suspected infection.
### The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients. 

In [ ]:
vitals_all_nosummary_72.head(2)

In [ ]:
#na values
vitals_all_nosummary_72.loc[
    vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[
    vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values


In [ ]:
print(
    vitals_all_nosummary_nonull['icustay_id'].nunique(), #14714/15207 unique icustay_id/icustay_id
)

In [ ]:
vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2',:]

#missingness_df.loc[missingness_df.loc[:,'label']=='SpO2',:]

In [ ]:
print((vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2','icustay_id'
        ].nunique()), #n=14668
    len(icustay_id)
     ) #15207

#print(14668/15207)

icustay_id_vitals = (vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'vitalid']=='SpO2','icustay_id'
        ].unique())

In [ ]:

vitals_all_nosummary_nonull_v=(vitals_all_nosummary_nonull.loc[
    vitals_all_nosummary_nonull.loc[:,'icustay_id']
    .isin(icustay_id_vitals),:]
)

In [ ]:
#lists and df with _v will be for patients who have atleast 1 SpO2 reading in first 3 days post t_0. 
vitals_all_nosummary_nonull_v
icustay_id_v= list(vitals_all_nosummary_nonull_v['icustay_id'].unique())
icustay_id_v= [int(x) for x in icustay_id_v]

In [ ]:
#updated final_pt_df2 for patients who have atleast 1 SpO2 reading in first 3 days post t_0. 
final_pt_df2_v = (
    final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_v),:]
)

In [ ]:
final_pt_df2_v.head()

In [ ]:
pd.crosstab(final_pt_df2['ab_course'], final_pt_df2['sc_result'])

In [ ]:
#todo, crosstab table of pts in new icustay_id_v
pd.crosstab(final_pt_df2_v['ab_course'], final_pt_df2_v['sc_result']) #n= 14668

In [ ]:
    ventcategory_df,
    vaso_dose_72,
    echodata_72,
    labs_all_nosummary_72,
    weightfirstday_df,
    heightfirstday_df,
    vitals_all_nosummary_72,
    uti_all_72,
    bg_all_nosummary_72,
    rrt_merged_allpt_df,
    gcs72_df,
    sofa_df_72

In [87]:
ventcategory_df

,icustay_id,t_0,vent_day1,vent_day2,vent_day3
0,267059.0,2182-11-12,Mech,None,None
1,278729.0,2159-11-08,Oxygen,Mech,None
2,219670.0,2190-08-01,Oxygen,None,None
3,295185.0,2154-11-11,Oxygen,None,None
4,211472.0,2193-12-11,None,None,None
5,259012.0,2120-05-24,None,None,None
7,252135.0,2166-04-13,Oxygen,Oxygen,Oxygen
8,293255.0,2167-10-21,Oxygen,None,None
9,261279.0,2167-10-26,None,None,None
10,274352.0,2132-07-28,Mech,None,None


time: 21.5 ms


In [96]:
#ventsettings_72=pd.read_csv(
#    wd+'/data/raw/csv/72_hr_window/%s_ventsettings_72.csv' %(date), index_col=0)

# date= '27082018' 
# ventsettings_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_ventsettings_72.csv' %(date)), index_col=0)

del(ventsettings_72)

time: 1.37 ms
